<h1>삼성전자(005930) 연결 재무상태표 DB에 넣기</h1>

In [4]:
import requests
from io import BytesIO
import pandas as pd
import numpy as np
import re
import os
import mysql.connector
from sqlalchemy import create_engine
from datetime import datetime

In [5]:
# opendart API로 상장사 사업보고서의 rcp_no 불러오기 
api_key = '27d86452b033fe973df6f1c980c37a2586801a1a'
corp_code = '00126380' # 삼성전자 corp_code
bgn_de = datetime.today().replace(year=2017).strftime('%Y%m%d')
end_de = datetime.today().strftime('%Y%m%d')

url = 'https://opendart.fss.or.kr/api/list.xml?crtfc_key={}&corp_code={}&bgn_de={}&end_de={}&last_reprt_at=Y&pblntf_detail_ty=A001&page_no=1&page_count=10'.format(api_key, corp_code, bgn_de, end_de)

In [6]:
resp = requests.get(url)
webpage = resp.content.decode('UTF-8')
webpage

'<?xml version="1.0" encoding="UTF-8" standalone="yes"?><result><status>000</status><message>정상</message><page_no>1</page_no><page_count>10</page_count><total_count>3</total_count><total_page>1</total_page><list><corp_code>00126380</corp_code><corp_name>삼성전자</corp_name><stock_code>005930</stock_code><corp_cls>Y</corp_cls><report_nm>사업보고서 (2018.12)</report_nm><rcept_no>20190401004781</rcept_no><flr_nm>삼성전자</flr_nm><rcept_dt>20190401</rcept_dt><rm>연</rm></list><list><corp_code>00126380</corp_code><corp_name>삼성전자</corp_name><stock_code>005930</stock_code><corp_cls>Y</corp_cls><report_nm>사업보고서 (2017.12)</report_nm><rcept_no>20180402005019</rcept_no><flr_nm>삼성전자</flr_nm><rcept_dt>20180402</rcept_dt><rm>연</rm></list><list><corp_code>00126380</corp_code><corp_name>삼성전자</corp_name><stock_code>005930</stock_code><corp_cls>Y</corp_cls><report_nm>사업보고서 (2016.12)</report_nm><rcept_no>20170331004518</rcept_no><flr_nm>삼성전자</flr_nm><rcept_dt>20170331</rcept_dt><rm>연</rm></list></result>'

In [7]:
rcp_no_list = re.findall(r'<rcept_no>(.*?)</rcept_no>', webpage)
period_list = re.findall(r'<report_nm>사업보고서 \((.*?)\)</report_nm>', webpage)

In [8]:
rcp_no_list

['20190401004781', '20180402005019', '20170331004518']

In [9]:
period_list

['2018.12', '2017.12', '2016.12']

In [10]:
# 각 rcp_no의 pair number인 dcm_no 조회
dcm_no_list = []
for rcp_no in rcp_no_list:
    resp = requests.get('http://dart.fss.or.kr/dsaf001/main.do?rcpNo={}'.format(rcp_no))
    webpage = resp.content.decode('utf-8')
    dcm_no = re.findall(r"'{}', '(.*?)',".format(rcp_no), webpage)[0]
    dcm_no_list.append(dcm_no)

In [11]:
dcm_no_list

['6616741', '6060273', '5540271']

In [12]:
# 파악된 rcp_no와 dcm_no 정보를 url에 넣어 재무제표 excel 파일을 pandas로 불러오기
user_agent = "Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36"

url = "http://dart.fss.or.kr/pdf/download/excel.do?rcp_no={}&dcm_no={}&lang=ko".format(rcp_no_list[0],dcm_no_list[0])
resp = requests.get(url, headers={"user-agent": user_agent})
table = BytesIO(resp.content)
df = pd.read_excel(table, sheet_name="연결 재무상태표", skiprows=6)

In [13]:
period_list.insert(0, 'account')

In [14]:
period_list

['account', '2018.12', '2017.12', '2016.12']

In [15]:
df.columns = period_list

In [16]:
df

,account,2018.12,2017.12,2016.12
0,자산,,,
1,유동자산,174697424,146982464,141429704
2,현금및현금성자산,30340505,30545130,32111442
3,단기금융상품,65893797,49447696,52432411
4,단기매도가능금융자산,,3191375,3638460
5,단기상각후원가금융자산,2703693,,
6,단기당기손익-공정가치금융자산,2001948,,
7,매출채권,33867733,27695995,24279211
8,미수금,3080733,4108961,3521197
9,선급금,1361807,1753673,1439938


In [17]:
# 계정명 내 공백제거
df['account'] = df['account'].str.strip()

In [18]:
df

,account,2018.12,2017.12,2016.12
0,자산,,,
1,유동자산,174697424,146982464,141429704
2,현금및현금성자산,30340505,30545130,32111442
3,단기금융상품,65893797,49447696,52432411
4,단기매도가능금융자산,,3191375,3638460
5,단기상각후원가금융자산,2703693,,
6,단기당기손익-공정가치금융자산,2001948,,
7,매출채권,33867733,27695995,24279211
8,미수금,3080733,4108961,3521197
9,선급금,1361807,1753673,1439938


In [19]:
# 공백란을 0로 채우기
df = df.replace(" ", 0)

In [20]:
df.astype({'account':'str',
           '2018.12':'int', 
           '2017.12':'int', 
           '2016.12':'int'})

,account,2018.12,2017.12,2016.12
0,자산,0,0,0
1,유동자산,174697424,146982464,141429704
2,현금및현금성자산,30340505,30545130,32111442
3,단기금융상품,65893797,49447696,52432411
4,단기매도가능금융자산,0,3191375,3638460
5,단기상각후원가금융자산,2703693,0,0
6,단기당기손익-공정가치금융자산,2001948,0,0
7,매출채권,33867733,27695995,24279211
8,미수금,3080733,4108961,3521197
9,선급금,1361807,1753673,1439938


In [21]:
df.dtypes

account    object
2018.12     int64
2017.12     int64
2016.12     int64
dtype: object

In [20]:
# 자산, 부채, 자본 항목 인덱싱

In [25]:
ale_tuple = ('자산', '부채', '자본')

bs_beg_idx = []
bs_end_idx = []

for val in ale_tuple:
    bs_beg_idx.append(df.loc[df['account'] == val,:].index[0] + 1)
    bs_end_idx.append(df.loc[df['account'] == val+'총계',:].index[0])
bs_end_idx

[27, 49, 60]

In [28]:
#connect MySQL
mydb = mysql.connector.connect(host = 'localhost',
                                user = 'root',
                                passwd = 'root',
                                db = 'fsdb')
mycursor = mydb.cursor()

In [32]:
ale_eng_tuple = ('bs_asset', 'bs_liability', 'bs_equity')

for beg, end, ale in zip(bs_beg_idx, bs_end_idx, ale_eng_tuple):
    # 자산, 부채, 자본항목별로 테이블 자르기
    table = df.loc[beg:end-1,:]
    
    # DB에 자산, 부채, 자본항목별 테이블 생성
    sql = """CREATE TABLE 005930_{}(
                    id bigint(20) unsigned NOT NULL AUTO_INCREMENT,
                    account varchar(255),
                    FY18 int(20),
                    FY17 int(20),
                    FY16 int(20),
                    PRIMARY KEY (id));""".format(ale)
    mycursor.execute(sql)
    
    # 각 테이블에 관련 계정명과 수치 입력
    for i in table.index:
        sql = """INSERT INTO 005930_{} (account, FY18, FY17, FY16) VALUES ("{}", {}, {}, {})""".format(ale, table['account'][i], table['2018.12'][i], table['2017.12'][i], table['2016.12'][i])
        mycursor.execute(sql)
    mydb.commit()

In [34]:
# 입력 확인
sql = "SELECT * FROM 005930_{}".format(ale_eng_tuple[1])
pd.read_sql_query(sql, mydb)

,id,account,FY18,FY17,FY16
0,1,유동부채,69081510,67175114,54704095
1,2,매입채무,8479916,9083907,6485039
2,3,단기차입금,13586660,15767619,12746789
3,4,미지급금,10711536,13899633,11525910
4,5,선수금,820265,1249174,1358878
5,6,예수금,951254,793582,685028
6,7,미지급비용,20339687,13996273,12527300
7,8,미지급법인세,8720050,7408348,2837353
8,9,유동성장기부채,33386,278619,1232817
9,10,충당부채,4384038,4294820,4597417


In [35]:
mydb.close()